# Experiment Notebook: System Metrics

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: Sanity Checks](#Analysis-1:-Sanity-Checks)
* [Analysis 2: Correlation Matrix](#Analysis-1:-Correlation-Matrix)
* [Analysis 3: PCV at Risk](#Analysis-3:-PCV-at-Risk)
* [Analysis 4: Capital Allocation Metrics](#Analysis-4:-Capital-Allocation-Metrics)
* [Appendix](#Appendix)

# Experiment Summary 

The purpose of this notebook is to demonstrate the system's standard metrics, KPIs and goals.

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from operator import lt, gt

from experiments.notebooks.visualizations.plotly_theme import cadlabs_colorway_sequence
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import itertools

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
color_cycle = itertools.cycle(cadlabs_colorway_sequence)

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Create a simulation for each analysis
simulation_0 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_2 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_3 = copy.deepcopy(default_experiment.experiment.simulations[0])

## Risk Analysis Methodology

The goal of the FEI Ecosystem Model Risk Analysis is to provide qualitative and quantitative recommendations to surface the most appropriate FEI monetary policy parameter settings across multiple scenarios. The risk analysis includes a cohesive set of model metrics, risk scores and protocol KPIs.

The main holistic risk analysis and parameter recommendation tool at our disposal is the FEI Capital Allocation model, where we allow the model to provide risk-weighted target allocations of FEI in all modeled avenues for FEI liquidity - LP, MM, FSD, Idle. These allocation targets will depend on parameter sweeps of FEI's monetary policy levers such as the FEI Savings Rate (see simplified ERD). The highest scoring parameter settings in terms of FEI Capital allocation and associated comparative KPIs will form the quantitative basis for recommendations.

It is to be noted that we can analyze KPIs comparatively based on monetary policy parameter settings in <b>Two ways</b> - qualitative/deterministic, and statistically based/stochastic.

Since the main driver of volatility in the FEI ecosystem is the Volatile Asset (an abstraction for Ethereum), we can model this volatile asset as Trajectory-based (based on a linear function), or based on a stochastic process such as a Geometric Brownian motion. The application of both settings for the Volatile asset allows us to respectively perform two analysis types.

1. Qualitative recommendations can be made as a result of comparative KPI analysis across different monetary policy parameter settings, using a trajectory model for the Volatile Asset. These do NOT involve monte carlo runs. The output of these analyses is to understand the impact on KPIs as a result of monetary policy changes for various final levels of Volatile Asset price. Ex: For a 30% VA price downturn, Stable backing ratio is higher with monetary policy 1 than policy 2, ie: Delta_1,2 Stable Backing ratio > 0, hence policy 1 is recommended.


2. Statistically-based recommendations can be made as a result of comparative KPI analysis across different monetary policy parameter settings, using a parameterized stochastic model for the Volatile Asset. These DO involve monte carlo runs. The output of these analyses is to construct a probability distribution for each KPI from which summary statistics can be derived. This allows us to empirically say that for a given parameter setting, KPIs are above or below key thresholds with a certain probability. Ex: Over 1000 simulations, 1-Day PCV at Risk is < 1M USD with a 90% probability with policy 1 and < 1M USD with an 85% probability with policy 2. Additionally, the statistical average PCVaR is lower with policy 1 than with policy 2, ie: Delta_1,2 avg. PCVaR > 0. Hence policy 1 is recommended.


# Analysis 0: PCV Sanity Checks

A simulation across 4 volatile asset price scenarios to illustrate PCV state evolution. Here, deterministic price trajectories for the Volatile Asset price are used, as opposed to parameterized stochastic processes.

In [ ]:
# Analysis-specific setup
simulation_0.model.params.update({
    "volatile_asset_price_process": [
        lambda _run, _timestep: 2_000,
        lambda _run, timestep: 2_000 if timestep < 365 / 4 else (1_000 if timestep < 365 * 3/4 else 2_000),
        lambda _run, timestep: 2_000 * (1 + timestep * 0.2 / 365),
        lambda _run, timestep: 2_000 * (1 - timestep * 0.2 / 365),
    ],
})

In [ ]:
# Experiment execution
df_1, exceptions = run(simulation_0)

## Post-processing and visualizations

In [ ]:
# Assign DataFrame for current analysis
df = df_1

In [ ]:
# Update subset labels
scenarios = {0: 'flat_price_trend', 1: 'step_price_trend', 2: 'bullish_price_trend', 3: 'bearish_price_trend'}
df['subset_label'] = df['subset'].map(lambda x: scenarios[x])

In [ ]:
fig = df.plot(y='volatile_asset_price', color='subset_label')

fig.update_layout(
    title="Sanity Check Volatile Asset Price Scenarios",
    xaxis_title="Timestamp",
    yaxis_title="Volatile Asset Price (USD)",
    autosize=False,
    width=1200,
    height=675,
)

fig.update_xaxes(title='Timestamp')

In [ ]:
fig = make_subplots(rows=5, cols=len(df.subset.unique()), shared_yaxes=True)

for subset in df.subset.unique():
    df_plot = df.query('subset == @subset')
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.volatile_asset_price,
            name="Volatile Asset Price",
            line=dict(color=cadlabs_colorway_sequence[0]),
            showlegend=(True if subset == 0 else False),
        ),
        row=1, col=subset+1,
    )

    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_pcv,
            name="Total PCV",
            line=dict(color=cadlabs_colorway_sequence[1]),
            showlegend=(True if subset == 0 else False),
        ),
        row=2, col=subset+1
    )

    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.collateralization_ratio,
            name="Collateralization Ratio",
            line=dict(color=cadlabs_colorway_sequence[2]),
            showlegend=(True if subset == 0 else False),
        ),
        row=3, col=subset+1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_volatile_asset_pcv,
            name="Volatile Asset PCV",
            line=dict(color=cadlabs_colorway_sequence[3]),
            showlegend=(True if subset == 0 else False),
            stackgroup='one',
        ),
        row=4, col=subset+1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_stable_asset_pcv,
            name="Stable Asset PCV",
            line=dict(color=cadlabs_colorway_sequence[4]),
            showlegend=(True if subset == 0 else False),
            stackgroup='one',
        ),
        row=4, col=subset+1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.liquidity_pool_tvl,
            name="Liquidity Pool TVL",
            line=dict(color=cadlabs_colorway_sequence[4]),
            showlegend=(True if subset == 0 else False),
        ),
        row=5, col=subset+1
    )


fig.update_layout(height=1000, title_text="Fei Protocol Model Sanity Checks")
fig.show()

# Analysis 1: Comparative KPI across Trajectories

In this analysis we wish to illustrate the usage of a trajectory-based volatile asset price for comparative analysis of a KPI or statistic of a KPI. In practice, we look at the effect of volatile asset price downturns at four levels: -10%- to -40% - each associated with a target stable PCV ration - on the stable PCV ratio at simulation end.

In [ ]:
pcv_ratio_settings = [0.9, 0.6, 0.4]

In [ ]:
# Analysis-specific setup
simulation_1.model.params.update({
    "volatile_asset_price_process": [
        lambda _run, timestep: (2_000 * (1 - timestep * 0.2 / 365)) if timestep < 365 / 4 else ((1_000 * (1 + timestep * 0.2 / 365)) if timestep < 365 * 3/4 else 2_000),
        lambda _run, timestep: (2_000 * (1 - timestep * 0.2 / 365)) if timestep < 365 / 4 else ((1_000 * (1 + timestep * 0.2 / 365)) if timestep < 365 * 3/4 else 2_000),
        lambda _run, timestep: (2_000 * (1 - timestep * 0.2 / 365)) if timestep < 365 / 4 else ((1_000 * (1 + timestep * 0.2 / 365)) if timestep < 365 * 3/4 else 2_000),
    ],
    "target_stable_pcv_ratio": pcv_ratio_settings,  # Target stable PCV ratio of 20% and 50%
    "target_stable_backing_ratio": [None],  # Disable stable backing ratio target
    "rebalancing_period": [int(365 / 4)],
})

In [ ]:
# Experiment execution
df_1, exceptions = run(simulation_1)

## Post-processing and visualizations

In [ ]:
# Assign DataFrame for current analysis
df = df_1

In [ ]:
# Update subset labels
n_scenarios = 3
scenarios = dict(zip(list(range(n_scenarios)), ['trajectory_'+str(x+1) for x in range(n_scenarios)]))
policy_settings = dict(zip(list(range(n_scenarios)), ['policy_setting_'+str(x) for x in pcv_ratio_settings]))

df['subset_label'] = df['subset'].map(lambda x: scenarios[x])
df['scenario_label'] = df['subset'].map(lambda x: policy_settings[x])

In [ ]:
fig = df.plot(y='volatile_asset_price', color='subset_label')

fig.update_layout(
    title="Volatile Asset Price Trajectories",
    xaxis_title="Timestamp",
    yaxis_title="Volatile Asset Price (USD)",
    autosize=False,
    width=1200,
    height=675,
)

fig.update_xaxes(title='Timestamp')

In [ ]:
fig2 = df.plot(y='collateralization_ratio', color='scenario_label')

fig2.update_layout(
    title="Collateralization Ratio",
    xaxis_title="Timestamp",
    yaxis_title="Collateralization Ratio",
    autosize=False,
    width=1200,
    height=675,
)

fig2.update_xaxes(title='Timestamp')

The difference of KPI matrix is constructed by taking the average of the KPI over a given window, and taking the pairwise difference of such KPI averages across trajectories.

In [ ]:
def generate_pairwise_rolling_corr(df, variable, window_size):
    L = []
    k = 0
    for i in range(3):
        for j in range(3):
            if i < j:
                d_ = df.query('subset==@i')[variable].rolling(window=window_size).corr(other=df.query('subset==@j')[variable])
                d_ = pd.DataFrame(d_)
                d_['subset'] = 'policy_setting_'+str(pcv_ratio_settings[i])+'_'+str(pcv_ratio_settings[j])
                k += 1
                L.append(d_)

    return pd.concat(L, axis=0)

In [ ]:
corr_series_df = generate_pairwise_rolling_corr(df, 'collateralization_ratio', 15)

In [ ]:
fig3 = corr_series_df.plot(y='collateralization_ratio', color='subset')

fig3.update_layout(
    title="15-Day Rolling Collateralization Ratio Correlation Series",
    xaxis_title="Timestamp",
    yaxis_title="Collateralization Ratio Pairwise Corrleation",
    autosize=False,
    width=1200,
    height=675,
)


In [ ]:
def display_delta_KPI(df, start_ts, end_ts, variable):
    
    # average of collateral ratio series KPI in set window
    avg_kpi = df.query('timestep >= @start_ts and timestep <= @end_ts').groupby('subset')[variable].mean()

    # create difference of averages matrix from series
    avg_kpi_delta = pd.DataFrame(np.column_stack([avg_kpi, np.subtract.outer(*[-avg_kpi.values]*2)]))
    avg_kpi_delta.columns = ['KPI'] + ['delta_KPI_'+str(x) for x in list(policy_settings.values())]
    avg_kpi_delta.index = list(scenarios.values())
    
    z_ = np.array(np.round(avg_kpi_delta.iloc[:,1:],4).to_numpy()),
    z__ = z_[0]
    z___ = np.triu(z_[0])
    z___[z___ == 0] = np.nan

    fig = ff.create_annotated_heatmap(
            z=z___,
            x=avg_kpi_delta.iloc[:,1:].columns.tolist(),
            y=avg_kpi_delta.iloc[:,1:].columns.tolist(),
            colorscale='rdpu',
            zmax=1, zmin=0,
            showscale=True,
        )
    
    fig.update_layout(
        title="Delta Collateral Ratio for each Policy Setting and Price Trajectory at Simulation End",
    )
    
    fig.show()

Taking a window further along the simulation, we see the results are qualitatively identical (due to lack of exogenous shocks in the dynamics) but qualitatively more accentuated

In [ ]:
start_ts = 365
end_ts = 365

display_delta_KPI(df, start_ts, end_ts, 'collateralization_ratio')

In the matrix above, generated by looking at the differences in average stable PCV ratio over the first 50 timesteps of the simulation, we see that by looking at the delta_CR columns across trajectory settings, simplistically, the greater the difference in volatile asset price downturn in any two scenarios, the greater the difference in stable PCV ratio. 

An even simpler conclusion is the average CR over the window is lower for greater levels of ETH downturn, with a difference between trajectories 1 and 4 (-10% and -40% respectively).

# Analysis 2: Correlation Matrix

In [ ]:
parameter_overrides = {
#     "target_stable_pcv_ratio": [0.5, 0.7],  # Target stable PCV ratio of 20% and 50%
#     "target_rebalancing_condition": [lambda a, b: lt(a, b)],  # Rebalance if less than target_stable_pcv_ratio
    "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
    "target_stable_pcv_ratio": [0.2, 0.5],  # Target stable PCV ratio of 20% and 50%
    "target_stable_backing_ratio": [None],  # Disable stable backing ratio target
    #"rebalancing_period": [int(365 / 4), int(365 / 4), int(365 / 12), int(365 / 12)],  # Rebalance quarterly vs monthly
}

# Analysis-specific setup
simulation_2.model.params.update(parameter_overrides)

In [ ]:
# Experiment execution
df_2, exceptions = run(simulation_2)

In [ ]:
# Post-processing and visualizations

In [ ]:
# Assign DataFrame for current analysis
df = df_2

In [ ]:
#np.round(df.query('subset == 0')[variables].corr().values,2).astype(str).tolist()

In [ ]:
variables = [
    "total_stable_asset_pcv",
    "total_pcv",
    "collateralization_ratio",
    "total_user_circulating_fei",
]

In [ ]:
z1 = df.query('subset == 0')[variables].corr()

fig = ff.create_annotated_heatmap(
            z=np.round(z1,2).to_numpy(),
            x=z1.columns.tolist(),
            y=z1.index.tolist(),
            colorscale='rdpu',
            zmax=1, zmin=-0.5,
            showscale=True,
            )

fig.update_layout(
    title="Correlations for Sanity Check State Variables - Policy 1",
#     xaxis_title="Timestamp",
#     yaxis_title="Volatile Asset Price (USD)",
    autosize=False,
    width=1200,
    height=675,
)

fig.show()

In [ ]:
z2 = df.query('subset == 1')[variables].corr()

fig2 = ff.create_annotated_heatmap(
            z=np.round(z2,2).to_numpy(),
            x=z2.columns.tolist(),
            y=z2.index.tolist(),
            colorscale='rdpu',
            zmax=1, zmin=-1,
            showscale=True,
            )

fig2.update_layout(
    title="Correlations for Sanity Check State Variables - Policy 2",
#     xaxis_title="Timestamp",
#     yaxis_title="Volatile Asset Price (USD)",
    autosize=False,
    width=1200,
    height=675,
)

fig2.show()

In [ ]:
def plot_states_vs_volatile_asset_price(df):
    fig_1 = px.scatter(df, x='volatile_asset_price', y='total_pcv', color='timestep')
    fig_1.update_layout(
        title="PCV vs. Volatile Asset Price",
        autosize=False,
        width=800,
        height=800,
    )
    fig_1.update_xaxes(title="Volatile Asset Price (USD)")
    fig_1.update_yaxes(title="PCV (USD)")
    fig_1.show()
    
    fig_2 = px.scatter(df, x='volatile_asset_price', y='collateralization_ratio_pct', color='timestep')
    fig_2.update_layout(
        title="Collateralization Ratio vs. Volatile Asset Price",
        autosize=False,
        width=800,
        height=800,
    )
    fig_2.update_xaxes(title="Volatile Asset Price (USD)")
    fig_2.update_yaxes(title="Collateralization Ratio (%)")
    fig_2.show()
    
    fig_3 = px.scatter(df, x='volatile_asset_price', y='total_user_circulating_fei', color='timestep')
    fig_3.update_layout(
        title="User-circulating FEI vs. Volatile Asset Price",
        autosize=False,
        width=800,
        height=800,
    )
    fig_3.update_xaxes(title="Volatile Asset Price (USD)")
    fig_3.update_yaxes(title="User-circulating FEI (USD)")
    fig_3.show()

In [ ]:
plot_states_vs_volatile_asset_price(df.query('subset == 0'))

In [ ]:
plot_states_vs_volatile_asset_price(df.query('subset == 1'))

# Analysis 3: PCV at Risk

## Analysis Summary

In this analysis, which is based on a stochastic Volatile Asset Price process with 10 realizations (10 monte carlo runs), we look at two policies (a parameter sweep of size 2) and look at the empirical probability of the PCV at Risk KPI being below a certain threshold, as well as comparatively examine the value of the KPI across both policies to yield a recommendation.

### Experiment Configuration

In [ ]:
from model.stochastic_processes import generate_volatile_asset_price_scenarios
df_price_scenarios = generate_volatile_asset_price_scenarios()
df_price_scenarios

In [ ]:
fig = df_price_scenarios.plot(y=['base_price_trend', 'bearish_price_trend', 'bullish_price_trend'])

fig.update_layout(
    title="Stochastic Volatile Asset Price Trend Scenarios",
    xaxis_title="Timestamp",
    yaxis_title="Volatile Asset Price (USD)",
    autosize=False,
    width=1200,
    height=675,
)

fig.show()

In [ ]:
fig = df_price_scenarios.plot(y=['base_price_volatility', 'low_price_volatility', 'high_price_volatility'])

fig.update_layout(
    title="Stochastic Volatile Asset Price Volatility Scenarios",
    xaxis_title="Timestamp",
    yaxis_title="Volatile Asset Price (USD)",
    autosize=False,
    width=1200,
    height=675,
)

fig.show()

In [ ]:
# Execute N Monte Carlo runs
simulation_3.runs = 6

# Construct Monte Carlo simulation of all stochastic price scenarios
volatile_asset_price_samples = [scenario for _key, scenario in df_price_scenarios.iteritems()]

In [ ]:
parameter_overrides = {
#     "volatile_asset_price_process": [
#         lambda run, timestep: volatile_asset_price_samples[run - 1][timestep]
#     ],  # Perform Monte Carlo simulation of all stochastic price scenarios above
    "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
    "target_stable_pcv_ratio": [0.2, 0.5],  # Target stable PCV ratio of 20% and 50%
    "target_stable_backing_ratio": [None],  # Disable stable backing ratio target
    #"target_rebalancing_condition": [lambda a, b: lt(a, b), lambda a, b: gt(a, b)],  # Rebalance if less than target_stable_pcv_ratio
    #"rebalancing_period": [int(365 / 4), int(365 / 4), int(365 / 12), int(365 / 12)],  # Rebalance quarterly vs monthly
}

# Update default model parameters and construct parameter sweep / parameter subsets
simulation_3.model.params.update(parameter_overrides)

### Experiment Execution

In [ ]:
# Experiment execution
df_3, exceptions = run(simulation_3)

In [ ]:
df_3['subset'].value_counts()

In [ ]:
df_3.groupby([
    'subset'
])[[
    'target_stable_pcv_ratio',
    'rebalancing_period',
]].last()

In [ ]:
# Update run labels
scenarios = dict(enumerate(df_price_scenarios))
df_3['run_label'] = df_3['run'].map(lambda x: scenarios[x - 1])

In [ ]:
df_3.query('subset == 0').plot(y="volatile_asset_price", color="run_label", labels=dict(value=df_price_scenarios.columns))

In [ ]:
df_3.query('subset == 0').plot(y="total_pcv", color="run_label")

In [ ]:
df_3.query("run_label == 'bearish_price_trend'").plot(y="stable_pcv_ratio", color='subset')

In [ ]:
df_3.query("run_label == 'bullish_price_trend'").plot(x='timestep', y="stable_pcv_ratio", color='subset')

## PCV at Risk Calculation Across All Subsets and Runs

In [ ]:
def calculate_VaR(df, state_variable, alpha, timesteps):
    results = pd.DataFrame()

    for simulation in df.simulation.unique():
        df_simulation = df.query("simulation == @simulation")
        for subset in df_simulation.subset.unique():
            df_subset = df_simulation.query("subset == @subset")
            for run in df_subset.run.unique():
                df_run = df_subset.query("run == @run")

                returns = df_run[state_variable].pct_change()
                final_value = df_run[state_variable].iloc[-1]
                q = returns.quantile(1 - alpha)
                value_at_risk = abs(final_value * q) * np.sqrt(timesteps)

                result = pd.DataFrame({'simulation': [simulation], 'subset': [subset], 'run': [run], 'VaR': [value_at_risk], 'q': [q]})
                results = pd.concat([results, result])

    return results.reset_index(drop=True)

In [ ]:
df_var = calculate_VaR(df_3, "total_pcv", alpha=0.95, timesteps=1)
df_var

In [ ]:
df_var.query("subset == 0")[["VaR", "q"]].describe()

In [ ]:
df_var_stats_0 = df_var.query("subset == 0")[["VaR", "q"]].describe()

In [ ]:
print(f"1-day average PCV at Risk at 95th quantile for subset 0: \n {df_var_stats_0['VaR'].loc['mean']:,.2f} USD")

In [ ]:
df_var_stats_1 = df_var.query("subset == 1")[["VaR", "q"]].describe()

In [ ]:
print(f"1-day average PCV at Risk at 95th quantile for subset 1: \n {df_var_stats_1['VaR'].loc['mean']:,.2f} USD")

## 1-day PCV at Risk for Policy 0

A visualization of PCVaR calculation for one monte carlo run for one Policy (parameter setting).

In [ ]:
def get_data_to_plot(df, run, subset):
    pcv_ret = df.query('run == @run and subset == @subset')['total_pcv'].pct_change()
    var = df_var.query('run == @run and subset == @subset')['VaR'].iloc[0]
    q = df_var.query('run == @run and subset == @subset')['q'].iloc[0]
    
    return pcv_ret, var, q

In [ ]:
fig = make_subplots(rows=6, cols=2,
                    x_title='PCV Daily Returns - Left: Policy 0, Right: Policy 1',
                    y_title='Number of Observations',
                   )

for subset in [0, 1]:
    for run in range(1,7):

        pcv_ret, var, q = get_data_to_plot(df_3, run, subset)

        fig.add_trace(
            px.histogram(pcv_ret, x="total_pcv", nbins=100).data[0],
            row=run, col=subset+1)

        fig.add_vline(x=q, row=run, col=subset+1)


fig.update_layout(
    title="Histogram of PCV Returns for Runs and Policy Settings",
    autosize=False,
    #width=1200,
    height=1600,
)

#fig.update_xaxes(xaxis_title='a')

fig.show()

print(f'1-Day PCVar for Run 1, Policy 0 (Subset 0) is {var:,.2f} USD with 5% quantile value {100*q:.2f}%')

## Probability of PCV at Risk Being Below Threshold

In [ ]:
quantile_return_threshold = -0.01


def calculate_VaR_threshold_probability(df, threshold):
    results = pd.DataFrame()
    
    for subset in df.subset.unique():
        df_subset = df.query("subset == @subset")
        
        df_threshold = df_subset["q"] <= threshold
        probability = df_threshold.sum() / len(df_threshold)
        
        result = pd.DataFrame({'subset': [subset], 'threshold': [threshold], 'probability': [probability]})
        results = pd.concat([results, result])
    
    return results.reset_index(drop=True)

In [ ]:
q_probabilities = calculate_VaR_threshold_probability(df_var, threshold=quantile_return_threshold)

In [ ]:
for subset in q_probabilities.subset.unique():
    print(f"""For Policy {subset + 1}, the 1-Day PCV at Risk is less than {abs(quantile_return_threshold*100):.2f}% with a {100*q_probabilities.query('subset == @subset')['probability'].iloc[0]:.2f}% probability""")

From this analysis, we see that over the 10 monte carlo runs for each policy (each subset), since the probability of PCV at risk being less than 2% of total PCV on any given day is lower for policy 1 than for policy 2, policy 1 does a better job at risk mitigation, hence we recommend policy 1.

## Comparative PCV at Risk

In [ ]:
avg_VaR_delta = df_var_stats_0['VaR'].loc['mean'] - df_var_stats_1['VaR'].loc['mean']
avg_VaR_quantile_delta = df_var_stats_0['q'].loc['mean'] - df_var_stats_1['q'].loc['mean']

In [ ]:
print(f"The Average PCVaR Delta between parameter for policies 1 and 2 is: \n {avg_VaR_delta:,.2f} USD")

In [ ]:
print(f"The Average PCVaR Quantile Delta between parameter for policies 1 and 2 is: \n {avg_VaR_quantile_delta:,.4f}")

We conclude that while the 1-Day PCVaR is greater for policy 1 than for policy 2, meaning more PCV is at risk on any given day at a 95% quantile, the value of this quantile is lower, meaning the PCV has a lower magnitude of negative returns, attesting to the risk mitigating effect of policy 1. Hence, policy 1 is recommended.

# Analysis 4: Capital Allocation Metrics

In [ ]:
fei_capital_allocation_variables = [
    'fei_idle_pcv_deposit_balance',
    'fei_liquidity_pool_pcv_deposit_balance',
    'fei_money_market_pcv_deposit_balance'
]
fei_capital_allocation_variables.sort()

In [ ]:
import plotly.express as px

px.area(df_2, y=fei_capital_allocation_variables, groupnorm='percent')

In [ ]:
df_allocations = df[fei_capital_allocation_variables].iloc[-1]

px.pie(df_allocations.sort_index(), values=df_allocations.values, names=df_allocations.index)

# Appendix

In [ ]:
# Assign DataFrame for current analysis
df = df_3.query('run == 1')

In [ ]:
_df = df.groupby(['subset','timestep']).mean().query('subset == 0')
stats_df = _df.describe()
stats_df.loc['skew'] = _df.skew()
stats_df.loc['kurtosis'] = _df.kurtosis()
# TODO: max drawdown & other relevant summary stats here

stats_df

In [ ]:
fig = df.plot(y="collateralization_ratio_pct", color='subset')
fig.add_hline(y=150)
fig.update_layout(
    title="Collateralization Ratio Over Time",
    xaxis_title="Timestamp",
    yaxis_title="Collateralization Ratio (%)",
    autosize=False,
    width=1200,
    height=675,
)
fig.show()

In [ ]:
fig = df.plot(y="stable_backing_ratio_pct", color='subset')
fig.add_hline(y=70)
fig.show()

In [ ]:
fig = df.plot(y="stable_pcv_ratio_pct", color='subset')
fig.add_hline(y=50)
fig.show()

In [ ]:
fig = df.plot(y="pcv_yield_rate_pct", color='subset')

fig.update_layout(
    title="PCV Yield Rate",
)
fig.update_xaxes(title="Timestamp")
fig.update_yaxes(title="PCV Yield Rate (%)")
fig.show()

In [ ]:
fig = df.plot(y="pcv_yield_ratio_pct")

fig.update_layout(
    title="PCV Yield Ratio",
)
fig.update_xaxes(title="Timestamp")
fig.update_yaxes(title="PCV Yield Ratio (%)")
fig.show()

In [ ]:
fig = df.plot(y="protocol_equity", color='subset')

fig.update_layout(
    title="Protocol Equity",
)
fig.update_xaxes(title="Timestamp")
fig.update_yaxes(title="Protocol Equity (USD)")
fig.show()